# 1. Installation and Imports

In [4]:
# Install required packages (if any)
# !pip install pandas

# Import libraries
from collections import defaultdict
import pandas as pd
import sys

# 2. Configuration Constants

In [5]:
# File configuration
INPUT_FILE = "Mov preparacion 02.2025.csv"
OUTPUT_FILE = "optimized_warehouse_locations.csv"

# Warehouse configuration
LOCATION_FILTER_SUFFIX = "10"  # Only process locations ending with this
EXCLUDED_LOCATION_PREFIX = "4"  # Exclude locations starting with this

# Brand conflict configuration
BRANDS_TO_SEPARATE = ["GRANINI", "JUVER", "MINUTE MAID"]
CONFLICT_DISTANCE = 2  # Minimum distance required between competing brands

# Zone sorting configuration (order of processing)
ZONE_SORTING_RULES = [
    {"prefix": "600", "max_zone": 48, "condition": "<="},
    {"prefix": "620", "max_zone": None, "condition": None},
    {"prefix": "660", "max_zone": 43, "condition": "<="},
    {"prefix": "680", "max_zone": 43, "condition": "<="},
    {"prefix": "600", "max_zone": 48, "condition": ">"},
    {"prefix": "630", "max_zone": None, "condition": None},
    {"prefix": "660", "max_zone": 43, "condition": ">"},
    {"prefix": "680", "max_zone": 43, "condition": ">"},
    {"prefix": "690", "max_zone": None, "condition": None}
]

# 3. Data Loading Functions

In [9]:
def load_and_validate_data(file_path):
    """
    Load CSV data with comprehensive error handling and validation.
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        return None
    except Exception as e:
        return None

    # Clean column names (remove leading/trailing spaces)
    df.columns = df.columns.str.strip()

    # Validate required columns exist
    required_columns = ["Material", "CtdTeóricaDesde", "Texto breve de material", "Ubic.proc."]
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        print(f"Error: Missing required columns: {missing_columns}")
        print(f"   Available columns: {list(df.columns)}")
        return None

    return df

def apply_initial_filters(df):
    """
    Apply business rule filters to the dataset.
    """
    initial_count = len(df)

    # Filter out locations starting with excluded prefix
    df = df[~df["Ubic.proc."].astype(str).str.startswith(EXCLUDED_LOCATION_PREFIX)]

    # Filter for locations ending with specified suffix
    df = df[df["Ubic.proc."].astype(str).str.endswith(LOCATION_FILTER_SUFFIX)]

    filtered_count = initial_count - len(df)

    return df

# 4. Data Processing Functions

In [13]:
def process_product_groups(df):
    """
    Group products by material and calculate aggregate statistics.
    """
    # Ensure quantity column is numeric
    df["CtdTeóricaDesde"] = pd.to_numeric(df["CtdTeóricaDesde"], errors="coerce")

    # Calculate total unique products
    total_products = df["Material"].nunique()

    # Create base data tuples (material, quantity, description, location)
    matrix_data = list(zip(
        df["Material"],
        df["CtdTeóricaDesde"],
        df["Texto breve de material"],
        df["Ubic.proc."]
    ))

    # Group by material using defaultdict for efficient aggregation
    grouped_data = defaultdict(lambda: [0, 0, "", ""])  # [total_qty, count, desc, loc]

    for material, quantity, description, location in matrix_data:
        if pd.notnull(quantity):
            grouped_data[material][0] += quantity  # Sum quantities
            grouped_data[material][1] += 1         # Count occurrences
            grouped_data[material][2] = description  # Store description
            grouped_data[material][3] = location     # Store location

    # Convert to list with calculated frequency (count/total_products)
    grouped_matrix = [
        (material, total, count / total_products, desc, loc)
        for material, (total, count, desc, loc) in grouped_data.items()
    ]

    # Sort by frequency (descending) and then by quantity (descending)
    grouped_matrix.sort(key=lambda x: (x[2], x[1]), reverse=True)

    # Extract original locations for sorting
    extracted_locations = [loc for _, _, _, _, loc in grouped_matrix]

    return grouped_matrix, total_products, extracted_locations

def parse_location(location):
    """
    Parse location string into components.
    """
    try:
        block, zone, subzone = location.split("-")
        return block, int(zone), subzone
    except (ValueError, IndexError):
        return "999", 999, "999"

# 5. Location Sorting Functions

In [12]:
def filter_and_sort_locations(location_list, prefix, max_value=None, condition=None):
    """
    Filter locations by prefix and zone conditions, then sort by zone.
    """
    result = []

    for location in location_list:
        if location.startswith(prefix):
            block, zone, subzone = parse_location(location)

            # Apply zone condition if specified
            if condition == '<=' and max_value is not None:
                if zone <= max_value:
                    result.append(location)
            elif condition == '>' and max_value is not None:
                if zone > max_value:
                    result.append(location)
            elif condition is None:
                result.append(location)

    # Sort by zone number
    return sorted(result, key=lambda x: parse_location(x)[1])

def apply_location_sorting(extracted_locations):
    """
    Apply custom sorting rules to locations based on warehouse zoning.
    """
    sorted_locations = []

    # Apply each sorting rule in configuration order
    for rule in ZONE_SORTING_RULES:
        filtered_locations = filter_and_sort_locations(
            extracted_locations,
            rule["prefix"],
            rule["max_zone"],
            rule["condition"]
        )
        sorted_locations.extend(filtered_locations)

    # Remove duplicates while preserving order
    unique_sorted_locations = list(dict.fromkeys(sorted_locations))

    return unique_sorted_locations

# 6. Conflict Resolution Functions

In [14]:
def contains_brand(description, brands):
    """
    Check if product description contains any of the specified brands.
    """
    if pd.isna(description):
        return False

    description_upper = str(description).upper()
    return any(brand.upper() in description_upper for brand in brands)

def detect_brand_conflicts(final_products):
    """
    Detect conflicts where competing brands are too close to each other.
    """
    conflicts = []

    for i in range(len(final_products)):
        current_material, _, _, current_description, current_location = final_products[i]

        # Skip if not a brand of interest
        if not contains_brand(current_description, BRANDS_TO_SEPARATE):
            continue

        current_block, current_zone, _ = parse_location(current_location)

        # Check neighbors within conflict distance
        for offset in [-2, -1, 1, 2]:
            j = i + offset
            if 0 <= j < len(final_products):
                neighbor_material, _, _, neighbor_description, neighbor_location = final_products[j]
                neighbor_block, neighbor_zone, _ = parse_location(neighbor_location)

                # Check if this is a conflict
                if (current_block == neighbor_block and
                    abs(current_zone - neighbor_zone) <= CONFLICT_DISTANCE and
                    contains_brand(neighbor_description, BRANDS_TO_SEPARATE)):

                    # Register conflict (store as sorted tuple to avoid duplicates)
                    conflict_pair = tuple(sorted((i, j)))
                    conflicts.append(conflict_pair)

    # Remove duplicate conflicts
    unique_conflicts = list(set(conflicts))

    if unique_conflicts:
        print(f"Detected {len(unique_conflicts)} brand conflicts requiring resolution")
    else:
        print("No brand conflicts detected")

    return unique_conflicts

def resolve_brand_conflicts(final_products, conflicts):
    """
    Resolve brand conflicts by swapping locations with non-competing products.
    """
    resolved_count = 0

    for i, j in conflicts:
        conflict_resolved = False

        # Try to resolve by swapping with neighbors of first conflict product
        for swap_offset in [-1, 1]:
            swap_index = i + swap_offset
            if (0 <= swap_index < len(final_products) and
                not contains_brand(final_products[swap_index][3], BRANDS_TO_SEPARATE)):

                # Swap locations only
                final_products[i][4], final_products[swap_index][4] = final_products[swap_index][4], final_products[i][4]
                conflict_resolved = True
                resolved_count += 1
                break

        if not conflict_resolved:
            # Try to resolve by swapping with neighbors of second conflict product
            for swap_offset in [-1, 1]:
                swap_index = j + swap_offset
                if (0 <= swap_index < len(final_products) and
                    not contains_brand(final_products[swap_index][3], BRANDS_TO_SEPARATE)):

                    # Swap locations only
                    final_products[j][4], final_products[swap_index][4] = final_products[swap_index][4], final_products[j][4]
                    resolved_count += 1
                    break

    if resolved_count > 0:
        print(f"Resolved {resolved_count} brand conflicts through location swapping")

    return final_products

# 7. Output Functions

In [16]:
def display_results(final_products, total_products):
    """
    Display formatted results of the warehouse optimization.
    """
    print("\n" + "="*90)
    print("WAREHOUSE LOCATION OPTIMIZATION RESULTS")
    print("="*90)
    print(f"Total different products: {total_products}\n")

    # Table header
    print("{:<12} {:<12} {:<15} {:<30} {:<15}".format(
        "Material", "Quantity", "Frequency", "Description", "Location"))
    print("-" * 85)

    # Display first 20 rows to avoid overwhelming output
    display_count = min(20, len(final_products))
    for i, (material, total, freq, desc, location) in enumerate(final_products[:display_count]):
        print("{:<12} {:<12.1f} {:<15.4f} {:<30} {:<15}".format(
            material, total, freq, desc[:28] + "..." if len(desc) > 28 else desc, location))

    if len(final_products) > display_count:
        print(f"... and {len(final_products) - display_count} more products")

    print("-" * 85)
    print("Optimization completed successfully")

def save_results_to_csv(final_products, output_file):
    """
    Save optimization results to CSV file.
    """
    try:
        result_df = pd.DataFrame(
            final_products,
            columns=["Material", "Quantity", "Frequency", "Description", "Location"]
        )
        result_df.to_csv(output_file, index=False)
        print(f"Results saved to '{output_file}'")

        # Show download link in Colab
        from google.colab import files
        files.download(output_file)

    except Exception as e:
        print(f"Error saving results: {e}")

# 8. Main Execution

In [18]:
print("🎯 WAREHOUSE OPTIMIZATION EXECUTION")
print("="*50)

# Step 1: Upload file first
print("Please upload your CSV file when prompted")
from google.colab import files
uploaded = files.upload()

# Get the uploaded filename
if uploaded:
    INPUT_FILE = list(uploaded.keys())[0]
    print(f"File '{INPUT_FILE}' uploaded successfully!")
else:
    print("No file uploaded. Using default filename.")

🎯 WAREHOUSE OPTIMIZATION EXECUTION
Please upload your CSV file when prompted


Saving Mov preparacion 02.2025.csv to Mov preparacion 02.2025 (1).csv
File 'Mov preparacion 02.2025 (1).csv' uploaded successfully!


# 9. Execute Data Processing

In [21]:
# Load and validate data
df = load_and_validate_data(INPUT_FILE)
if df is None:
    print("Cannot continue without valid data")
else:
    # Apply initial filters
    df = apply_initial_filters(df)

    # Process product groups
    grouped_matrix, total_products, extracted_locations = process_product_groups(df)

# 10. Execute Location Assignment

In [22]:
if 'grouped_matrix' in locals() and 'extracted_locations' in locals():
    # Apply location sorting
    sorted_locations = apply_location_sorting(extracted_locations)

    # Assign sorted locations to products
    final_products = []
    for (material, total, frequency, desc, _), new_location in zip(grouped_matrix, sorted_locations):
        final_products.append([material, total, frequency, desc, new_location])

    print(f"Assigned locations to {len(final_products)} products")
else:
    print("Required variables not found. Please run previous blocks first.")

Assigned locations to 370 products


# 11. Execute Conflict Resolution

In [25]:
if 'final_products' in locals():
    # Detect and resolve brand conflicts
    conflicts = detect_brand_conflicts(final_products)
    if conflicts:
        final_products = resolve_brand_conflicts(final_products, conflicts)
else:
    print("Final products not found. Please run previous blocks first.")

Detected 3 brand conflicts requiring resolution
Resolved 3 brand conflicts through location swapping


# 12. Final Results and Download

In [24]:
if 'final_products' in locals() and 'total_products' in locals():
    # Display results
    display_results(final_products, total_products)

    # Save to CSV and download
    save_results_to_csv(final_products, OUTPUT_FILE)
else:
    print("Required variables not found. Please run previous blocks first.")


WAREHOUSE LOCATION OPTIMIZATION RESULTS
Total different products: 381

Material     Quantity     Frequency       Description                    Location       
-------------------------------------------------------------------------------------
0RF0161      9475.0       1.4961          COCACOLA LATA33 C24 (SIN HC8... 600-004-10     
0LT0235      1692.0       1.4803          LA LEVANTINA AVENA ESP.HOST ... 600-006-10     
0RF0187      10777.0      1.4593          COCACOLA ZER LATA33 C24 (SIN... 600-007-10     
0LT0090      1148.0       1.2756          LETONA SEMI SIN LACTOSA 1L P... 600-008-10     
ED13LT       1594.0       1.2598          ESTRELLA DAMM 1/3 LATA         600-009-10     
EC13P6       929.0        1.1339          DAURA DAMM 1/3 SR PACK CESTA... 600-011-10     
0LT0034      2542.0       1.0420          LETONA SEMI ESPEC HOSTELERIA... 600-012-10     
0ZU0020      686.0        1.0184          GRANINI MELOCOTON  20CL 24U    600-013-10     
0ZU0024      680.0        1.0105   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>